In [5]:
# Currency exchange using openexchangerates.org

import os
import requests
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

API_KEY = os.environ.get('OPX_KEY')

r = requests.get(
    'https://openexchangerates.org/api/latest.json',
    params = {
        'app_id': API_KEY,
        'symbols': 'CAD,USD',
        'show_alternative': 'true'
    }
)

rates_ = r.json()['rates']
rates_


{'CAD': 1.330932, 'USD': 1}

In [6]:

# Calculate CAD to USD
symbol_from = 'CAD'
symbol_to = 'USD'
value = 3000

value * 1/rates_.get(symbol_from)*rates_.get(symbol_to)

2254.059561269847

In [16]:
# Encapsulate the code into a Class

class CurrencyConverter:
    def __init__(self, symbols, API_KEY):
        self.API_KEY = API_KEY
        self.symbols = symbols
        self._symbols = ','.join([str(s) for s in symbols])
        r = requests.get(
            'https://openexchangerates.org/api/latest.json',
            params = {
                'app_id': self.API_KEY,
                'symbols': self._symbols,
                'show_alternative': 'true'
                }
            )
        self.rates_ = r.json()['rates']
        self.rates_['USD'] = 1
    def convert(self, value, symbol_from, symbol_to, round_output=True):
        try:
            x = (value
                * 1/self.rates_.get(symbol_from)
                * self.rates_.get(symbol_to))
            if round_output:
                return round(x, 2)
            else:
                return x
        except TypeError:
            print('Unavailable or invalid symbol')
            return None


In [17]:
# Use the currency converter class
API_KEY = os.environ.get("OPX_KEY")
c = CurrencyConverter(['CAD','USD'], API_KEY)

print(c.convert(3000, 'CAD', 'USD'))
print(c.convert(5000, 'USD', 'CAD'))

2254.68
6652.84


In [18]:
# Show alternative currency rates
c = CurrencyConverter(['CAD', 'USD', 'DOGE', 'ETH', 'BTC'], API_KEY)
c.rates_

{'BTC': 9.578106e-05,
 'CAD': 1.330567,
 'DOGE': 382.947504,
 'ETH': 0.002920646,
 'USD': 1}

In [20]:
# Apply currency class to income schedule
import pandas as pd

df = pd.read_excel('../data/xirr.xlsx', sheet_name="irregular")
df['total'] = df.income + df.expenses
df

,date,income,expenses,total
0,2017-01-01,40,-3000,-2960
1,2017-01-25,40,-50,-10
2,2017-02-12,80,-50,30
3,2017-02-14,100,-30,70
4,2017-03-04,100,-20,80
5,2017-04-23,160,-30,130
6,2017-05-07,140,-20,120
7,2017-05-21,140,-40,100
8,2017-06-04,80,-40,40
9,2017-06-19,180,-30,150


In [21]:
#Convert everything at once
df['total'].apply(lambda x: c.convert(x, 'CAD', 'USD'))

0    -2224.62
1       -7.52
2       22.55
3       52.61
4       60.12
5       97.70
6       90.19
7       75.16
8       30.06
9      112.73
10     240.50
11      97.70
12     165.34
13     278.08
14     285.59
15     225.47
16     240.50
17     375.78
Name: total, dtype: float64